use current app predict the next app

### 1. Reading data

In [1]:
import pandas as pd

In [2]:
dat6 = pd.read_csv('dat6.csv')

In [10]:
# an example data of a user (id = 1)
i = 3
data = dat6.loc[dat6.newid == i, :]

### 2. Function to read each id data, give the result is accuracies of the model to predict napps (1, 2, 3, 4, 5)

In [33]:
def LU1(data):
    
    from random import sample, seed

    # 1 prepare train and test data
    
    data1 = data.sort_values('startTimeMillis')   
    data2 = data1.groupby('ids').count()
    data2.reset_index(level = 0, inplace = True)
    ses2 = data2.loc[data2.startTime > 1,:]     
    ses2_set = set(ses2.ids)   
    data2s = data1.loc[data1.ids.isin(ses2_set)]        
    list_apps = list(data2s.app_name.unique())
    
    # Cross validation k = 5
    n = len(ses2_set)
    k=5
    m = n//k
    
    train = []
    test = []
    left = ses2_set.copy()

    for i in range(5):
        if i < 4:
            seed(123)
            tst = set(sample(left, m))
            trn = ses2_set - tst

            train_data = data2s.loc[data2s.ids.isin(trn)]
            test_data = data2s.loc[data2s.ids.isin(tst)]

            train.append(train_data)
            test.append(test_data)

            left -= tst
        else:
            tst = left
            trn = ses2_set - tst

            train_data = data2s.loc[data2s.ids.isin(trn)]
            test_data = data2s.loc[data2s.ids.isin(tst)]

            train.append(train_data)
            test.append(test_data)
       
    
    # 2 list of functions needed

    # function count the walks from this to that in the list_walks
    def count_walks(this, that):
        r = 0
        for walks in list_walks:
            n = len(walks)
            for i in range(n-1):           
                if (walks[i] == this) & (walks[i+1] == that):
                    r +=1
        return r

    # function predict the next app with the highest counting
    def next_apps(app, napps):

        total_walks = [count_walks(app, p) for p in list_apps]

        dt = pd.DataFrame(
        {'next_app': list_apps,
         'total_walks': total_walks
         })
        dt = dt.sort_values('total_walks', ascending=False)         

        if len(dt) == 0:
            return []   
        elif len(dt) == 1:
            result = [dt.iloc[0,0]]
        elif len(dt) == 2:
            result = list(dt.iloc[:2,0])
        elif len(dt) == 3:
            result = list(dt.iloc[:3,0])
        elif len(dt) == 4:
            result = list(dt.iloc[:4,0])
        else:
            result = list(dt.iloc[:5,0])

        ind = min(napps, len(result))

        return result[:ind]

    
    # using the re_dict to check the test set
    def hit_rate_result(re_dict, list_walks_test):
        total, hit = 0, 0

        for walks in list_walks_test:
            total += len(walks) - 1

            for i, walk in enumerate(walks[:-1]): 
                if walk in re_dict:
                    if walks[i+1] in re_dict[walk]:
                        hit += 1
        return round(hit/total,4)


    # 3. Prediction Accuracy

    total_hit_rate = [0, 0, 0, 0, 0] # to keep the accuracy of LU1 model to predict number of apps

    for i in range(len(train)): 

        data_train = train[i]
        ses2_list = list(data_train.ids.unique())

        list_walks = [] # keep the list of walks training
        for s in ses2_list:

            d = data_train.loc[data_train['ids'] == s,]
            list_walks.append(list(d['app_name'])) 
        
        # dictionary to keep the result of model training
        re_dict_l = [{app: next_apps(app, napps) for app in list_apps} for napps in range(1,6)]

        data_test = test[i]
        ses_list_test = list(data_test.ids.unique())

        list_walks_test = [] # keep the list of walks testing
        for s in ses_list_test:

            d = data_test.loc[data_test['ids'] == s,]
            list_walks_test.append(list(d['app_name'])) 

        for i in range(5):
            total_hit_rate[i] += hit_rate_result(re_dict_l[i], list_walks_test)

    return [round(total_hit_rate[i] /len(train), 4) for i in range(5)]


In [ ]:
import time
start = time.time() 

columns = [ 'LU1_1', 'LU1_2', 'LU1_3', 'LU1_4', 'LU1_5']
Result = pd.DataFrame(index = range(1,2010), columns = columns)

import time
start = time.time()

for i in range(1,2010):
    data = dat6.loc[dat6.newid == i, :]
    
    accuracy = LU1(data)
     
    for j in range(5):
        Result.iloc[i-1, j] = accuracy[j]

Result.to_excel('LU1_2009.xlsx')

end = time.time() 
print(end - start) 